In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

%matplotlib inline

import io, os, glob, gc
import timeit
from nbformat import current

HISTOGRAM_BITS_PER_COLOR = 2

def execute_notebook(nbfile):
    
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    
    ip = get_ipython()
    
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)

execute_notebook('cuav.ipynb')

ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_SUGGESTED_THRESHOLD = 35

SHOW_DUPES = True
MIN_DEDUPE_REGIONS = 2
#DEDUPE_REGIONS = False
#DDUPE_ACCURACY_BY = 10
#MIN_REGION_SCORE = 500

/home/johnchu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/johnchu/anaconda2/lib/python2.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """)
/home/johnchu/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
def ProcessFile(image):
        bin = InitBin(image)
        lut = InitLUT(bin)
        qimage = quantize(image, lut)

#         hist = BuildHistogram(qimage)
#         mask = HistogramThreshold(qimage, hist)
#         print 'Histogram Count Threshold: %i' % HISTOGRAM_COUNT_THRESHOLD

#         regions = AssignRegions(mask)
#         ScoreRegions(regions, qimage, hist)
        
#         print "Total Regions Before Pruning: %i\n" % regions.num_regions

        regions = Regions(qimage)
        
        regions.PruneRegionsByThreshold()
        regions.PruneRegionsBySpectra()
#        regions.PruneRegionsByMean()
#        regions.PruneRegionsBySpectra()
        regions.PruneRegionsByDupe()
        regions.PruneRegionsByProximity()
        
        del bin,lut,qimage #,hist,mask
        gc.collect()
        
        return regions

def LoadAnnotations(path):
    annotations = {}
    f = os.path.join(path, 'annotation.txt')

    alr = AnnotationsLR()
    if os.path.isfile(f):
        alr.jsonDeserializer(f)
        for key, notes in alr.annotations.iteritems():
            for pos in notes:
                notes = []
                if key in annotations:
                    notes = annotations[key]

                notes.append(pos)
                annotations[key] = notes
            
    return alr

In [3]:
#from guppy import hpy
#hp = hpy()
#before = hp.heap()

## critical section here
#raw = cv2.imread('batch/few/1.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
#regions = ProcessFile(image)

#after = hp.heap()
#leftover = after - before
##import pdb; pdb.set_trace()

#print leftover.byclass

In [4]:
#raw = cv2.imread('batch/od/image_car_full.png')
#image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)

#%timeit ProcessFile(image)
#%prun regions = ProcessFile(image)

In [5]:
#for dirname, dirnames, filenames in os.walk('batch/few/'):
#    for filename in sorted(filenames):
#        print(os.path.join(dirname, filename))


In [6]:
PrintSettings()

#raw_files_dir = 'batch/test/'
#raw_files_dir = '/home/johnchu/workspace/oddiagnostic/batch/images/ObjectDetection/camera/spectral/'
raw_files_dir = 'batch/test/'
proc_files_dir = raw_files_dir + 'results/'

if not os.path.exists(proc_files_dir):
    os.makedirs(proc_files_dir)
    
import cPickle
import pandas as pd
with open('od_classifier.pkl', 'rb') as fid:
    model = cPickle.load(fid)
# od_df = pd.read_csv('od_df.csv')
# od_df = od_df.drop(['Region Active','Positive'],axis=1)
# prediction = model.predict(od_df)
# print "validate model: ", prediction

alr = LoadAnnotations(raw_files_dir)

for f in sorted(glob.glob(raw_files_dir + '*.*')):
    file = f.replace(raw_files_dir,'')
    if file.lower().endswith('.jpg') or file.lower().endswith('.png'):
        print file
        raw = cv2.imread(f)
        image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
        
        regions = ProcessFile(image)
        
        regions.Save(proc_files_dir, file)
        
        print '\n'
        print 'Annotating Active Regions: %i' % regions.num_regions
        pruned_image = np.copy(image)
        RegionPredictor(model, regions)
        MarkRegions(pruned_image, regions)
        
        notes = None
        if file in alr.annotations:
            notes = alr.annotations[file]
            print notes
            MarkAnnotations(pruned_image, notes, int(alr.width), int(alr.height))
            
        bgr = cv2.cvtColor(pruned_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(proc_files_dir + file, bgr)
        print '\n'
    else:
        print 'Skipping %s' % file

Adaptive Threshold Disabled with count set at 35 

HISTOGRAM_BITS_PER_COLOR = 2
NUM_BINS = 4
COLOR_SPECTRUM = 65
ADAPTIVE_HISTOGRAM_THRESHOLD = False
HISTOGRAM_CEILING_THRESHOLD = 1500
HISTOGRAM_FLOOR_THRESHOLD = 50
SHOW_DUPES = True
DEDUPE_REGIONS = True
MIN_DEDUPE_REGIONS = 2
MIN_REGION_SCORE = 0

Skipping annotation.txt
Skipping annotation.txt~
image_car_full.png
InitBin:
image height=2048,width=2448,depth=3
min_r=0, max_r=255, bin_spacing.r=64
min_g=0, max_g=255, bin_spacing.g=64
min_b=34, max_b=255, bin_spacing.b=56

REGION_MERGE_PIXEL_DISTANCE = 20
Average Image Pixel: 42
Histogram Count Threshold: 39
Prune Common Regions (below threshold):
  Min Score: 0
  Regions Above Threshold (Active): 4
Prune Regions in Neighborhood:
  Neighborhood: 50
  Regions Above Neighborhood (Active): 1
  [id]: [1]
Prune Duplicated Regions:
  Accuracy: 1
  Regions Deduped (Active): 1
  [id]: [1]
Pruned Clustered Regions:
  Radius of Separation: 200
  Regions Isolated (Active): 1
  [id]: [1]


Annotati

In [10]:
%sx ipython nbconvert "oddiagnostic.ipynb"

['[NbConvertApp] Converting notebook oddiagnostic.ipynb to html',
 '[NbConvertApp] Writing 228093 bytes to oddiagnostic.html']

In [11]:
%sx mv "oddiagnostic.html" $proc_files_dir

[]

In [9]:
# Used to profile 
#
# raw = cv2.imread('batch/od/image_car_full.png')
# image = cv2.cvtColor(raw, cv2.COLOR_BGR2RGB)
# bin = InitBin(image)
# lut = InitLUT(bin)
# qimage = quantize(image, lut)

# %timeit regions = Regions(qimage)
